# AsyncSeq (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

Installing Packages Expecto.....

Installing Packages Expecto......

Installed Packages Expecto, 10.1.0

In [ ]:
#r "nuget:FSharp.Control.AsyncSeq"
#r "nuget:System.Reactive,5.0.0"
#r "nuget:System.Reactive.Linq,6.0.0-preview.1"

Installing Packages FSharp.Control.AsyncSeq System.Reactive System.Reactive.Linq

Installing Packages FSharp.Control.AsyncSeq. System.Reactive. System.Reactive.Linq.

Installing Packages FSharp.Control.AsyncSeq.. System.Reactive.. System.Reactive.Linq..

Installing Packages FSharp.Control.AsyncSeq... System.Reactive... System.Reactive.Linq...

Installing Packages FSharp.Control.AsyncSeq.... System.Reactive.... System.Reactive.Linq....

Installing Packages FSharp.Control.AsyncSeq..... System.Reactive..... System.Reactive.Linq.....

Installing Packages FSharp.Control.AsyncSeq...... System.Reactive...... System.Reactive.Linq......

Installing Packages FSharp.Control.AsyncSeq....... System.Reactive....... System.Reactive.Linq.......

Installed Packages FSharp.Control.AsyncSeq, 3.2.1 System.Reactive, 5.0.0 System.Reactive.Linq, 6.0.0-preview.1

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs

In [ ]:
open Common

## subscribeEvent

In [ ]:
let subscribeEvent (event: IEvent<'H, 'A>) map =
    let observable = System.Reactive.Linq.Observable.FromEventPattern<'H, 'A>(event.AddHandler, event.RemoveHandler)
    System.Reactive.Linq.Observable.Select (observable, fun event -> map event.EventArgs)
    |> FSharp.Control.AsyncSeq.ofObservableBuffered

In [ ]:
//// test

type TestEvent () as self =
    member val Calls = [] with get, set
    member val Event = Event<ErrorEventHandler, ErrorEventArgs> () with get

    member _.AddCall text =
        self.Calls <- self.Calls @ [ text ]

    member _.EventInterface =
        { new IEvent<ErrorEventHandler, ErrorEventArgs> with
            member _.AddHandler handler =
                self.AddCall "AddHandler"
                self.Event.Publish.AddHandler handler

            member _.RemoveHandler handler =
                self.AddCall "RemoveHandler"
                self.Event.Publish.RemoveHandler handler

            member _.Subscribe observer =
                self.AddCall "IObservable.Subscribe"
                let disposable = self.Event.Publish.Subscribe observer
                Object.newDisposable (fun () ->
                    self.AddCall "IObservable.Dispose"
                    disposable.Dispose ()
                )
        }

    member _.Subscribe () =
        subscribeEvent
            self.EventInterface
            (fun args ->
                let result = args.GetException().Message
                self.AddCall $"TestEvent.Subscribe({result})"
                result
            )

    member _.Iter subscription =
        subscription
        |> FSharp.Control.AsyncSeq.iteriAsync (fun i message -> async {
            self.AddCall $"TestEvent.Iter({i}:{message})"
        })

    member _.WaitCall text = async {
        while self.Calls |> List.last <> text do
            do! Async.SwitchToThreadPool ()
    }

In [ ]:
//// test

let testEvent = TestEvent ()

async {
    testEvent.AddCall "1"
    let! child = testEvent.Subscribe () |> testEvent.Iter |> Async.StartChild
    do! testEvent.WaitCall "AddHandler"
    testEvent.AddCall "2"
    do! child
    testEvent.AddCall "3"
}
|> Async.runWithTimeout 100
|> _equal None

testEvent.Calls
|> Seq.toList
|> _equal [ "1"; "AddHandler"; "2"; "RemoveHandler" ]

19:44:20 #1 [Debug] runWithTimeout / timeout: 100 / exception: The operation has timed out.
<null>
[ 1, AddHandler, 2, RemoveHandler ]


In [ ]:
//// test

let testEvent = TestEvent ()

async {
    testEvent.AddCall "1"
    let! child = testEvent.Subscribe () |> testEvent.Iter |> Async.StartChild
    do! testEvent.WaitCall "AddHandler"
    testEvent.AddCall "2"
    use _ = testEvent.EventInterface.Subscribe (fun args ->
        testEvent.AddCall $"testEvent.EventInterface.Subscribe({args})"
    )
    testEvent.AddCall "3"
    do! child
    testEvent.AddCall "4"
}
|> Async.runWithTimeout 100
|> _equal None

testEvent.Calls
|> _equal [ "1"; "AddHandler"; "2"; "IObservable.Subscribe"; "3"; "RemoveHandler"; "IObservable.Dispose" ]

19:44:20 #2 [Debug] runWithTimeout / timeout: 100 / exception: The operation has timed out.
<null>
[ 1, AddHandler, 2, IObservable.Subscribe, 3, RemoveHandler, IObservable.Dispose ]


In [ ]:
//// test

let testEvent = TestEvent ()

async {
    testEvent.AddCall "1"
    let! child = testEvent.Subscribe () |> testEvent.Iter |> Async.StartChild
    do! testEvent.WaitCall "AddHandler"
    testEvent.AddCall "2"
    use _ = testEvent.EventInterface.Subscribe (fun args ->
        async {
            do! testEvent.WaitCall "TestEvent.Iter(0:error)"
            testEvent.AddCall $"testEvent.EventInterface.Subscribe({args.GetException().Message})"
        }
        |> Async.RunSynchronously
    )
    testEvent.AddCall "3"
    testEvent.Event.Trigger (null, ErrorEventArgs (Exception "error"))
    testEvent.AddCall "4"
    do! child
    testEvent.AddCall "5"
}
|> Async.runWithTimeout 100
|> _equal None

testEvent.Calls
|> _equal [
    "1"
    "AddHandler"
    "2"
    "IObservable.Subscribe"
    "3"
    "TestEvent.Subscribe(error)"
    "TestEvent.Iter(0:error)"
    "testEvent.EventInterface.Subscribe(error)"
    "4"
    "RemoveHandler"
    "IObservable.Dispose"
]

19:44:20 #3 [Debug] runWithTimeout / timeout: 100 / exception: The operation has timed out.
<null>
[ 1, AddHandler, 2, IObservable.Subscribe, 3, TestEvent.Subscribe(error), TestEvent.Iter(0:error), testEvent.EventInterface.Subscribe(error), 4, RemoveHandler, IObservable.Dispose ]
